In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

right_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_right_fulltext.csv')
right_dataset = right_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
right_dataset = right_dataset.train_test_split(test_size=0.2)

# Prepocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text_cleaned"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_right_dataset = right_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=right_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4251 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1219 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1400 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1125 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2730 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1159 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1492 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (9533 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_right_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
#finetune the pretrained model on the dataset

training_args = TrainingArguments(
    output_dir="distillgpt2_right_fulltext_cleaned_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/tobijen/distillgpt2_right_fulltext_cleaned_torch into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.447900,3.329940
2,3.350700,3.303256
3,3.301500,3.288772
4,3.273000,3.282281
5,3.242900,3.281035
6,3.220800,3.279794


TrainOutput(global_step=23970, training_loss=3.3132392867386318, metrics={'train_runtime': 2621.2151, 'train_samples_per_second': 73.157, 'train_steps_per_second': 9.145, 'total_flos': 6263283111690240.0, 'train_loss': 3.3132392867386318, 'epoch': 6.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 26.57


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/312M [00:00<?, ?B/s]

Upload file runs/Aug01_14-11-59_27a4e2a2526d/events.out.tfevents.1690899130.27a4e2a2526d.2561.0:   0%|        …

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/distillgpt2_right_fulltext_cleaned_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
# generate text with the pipeline module from the transformers library
from transformers import pipeline

generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/distillgpt2_right_fulltext_cleaned_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Gay marriage is now part of federal court cases in some states.The White House, which has said the Obama administration has reached an agreement on the issue, said on Monday that it is reviewing the case for a possible court ruling on that issue.The lawsuit, which also seeks to bar the Obama administration from using contraception at birth, is among more than 80 lawsuits brought before the Supreme Court last year.The Obama administration also filed two separate lawsuits seeking to overturn Obamacare’s protections for same-sex marriage in 20'}]
social_2  =>  Abortion is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Abortion is one of the many stories of these cases that the courts have rejected. It's in this case because it is almost certainly the only abortion case in history. And it's also in this case because it happens so often that the courts have given consent to abortion. The right to abortions. And so there is an absolute right. It's because there are no restrictions or laws prohibiting abortion in state hospitals where one may have a baby. But it's also because there aren't a lot of exceptions to the"}]
education_1  =>  Public education is


...

[{'generated_text': 'Charter schools are among the fastest growing states to receive the federal government’s “super-majority” education aid.States received 15 percent of those funds by Christmas last year, while states in the “Medicare Act” enacted by President Donald Trump have received 17 percent.The funding bill, which seeks to provide money for federal education through 2020, seeks to “pay for the costs incurred by state, district and municipality in order to implement and subsidize the federal government’s educational'}]
economic_1  =>  The increase of taxes is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The increase of taxes is part of a renewed campaign against the wealthy, which Trump said he wants tax hikes. He said he wanted a higher revenue threshold and a repeal of the Affordable Care Act for the middle class.The U.S. has seen a decline in inflation since early February, and for the first time since March 2012, the value of goods rose almost twice it was three years ago.The Federal Reserve has raised rates for all U.S. real and business assets this time, but it has struggled in recent'}]
economic_2  =>  Government regulations on businesses are


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Government regulations on businesses are subject to the “expose-personality” restriction.The “expose-personality” ruling could impact businesses and individuals who want to apply for unemployment benefits to be reinstated and people who wish to apply for benefits to stay at work.The case was joined by several other states’ legal scholars. Attorney General Eric Holder has said the ruling puts the Biden administration into serious disrepute even more.“The U.S. Supreme Court has long acted in a'}]


In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is now part of federal court cases in some states.The White House, which has said the Obama administration has reached an agreement on the issue, said on Monday that it is reviewing the case for a possible court ruling on that issue.The lawsuit, which also seeks to bar the Obama administration from using contraception at birth, is among more than 80 lawsuits brought before the Supreme Court last year.The Obama administration also filed two separate lawsuits seeking to overturn Obamacare’s protections for same-sex marriage in 20',
 'social_2': "Abortion is one of the many stories of these cases that the courts have rejected. It's in this case because it is almost certainly the only abortion case in history. And it's also in this case because it happens so often that the courts have given consent to abortion. The right to abortions. And so there is an absolute right. It's because there are no restrictions or laws prohibiting abortion in state hospitals where on

In [ ]:
# add the generated text to a json file, wich is used for the evaluation

import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "distilgpt2_right_fulltext_cleaned": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)